# Flaw Engine: FINAL FIX Cloud Trainer ♟️🚀

### 🏁 Step 1: Clone
Run this to get the code.

In [ ]:
import os
URL = "https://github.com/Gideon-gg101/flaw-engine.git"
ROOT = "/content/flaw_worker"
!rm -rf {ROOT}
!git clone {URL} {ROOT}
if os.path.exists(ROOT): print("\n✅ SUCCESS!")

### 🏁 Step 2: Build
Run this to compile the engine.

In [ ]:
import os, shutil, glob
!apt install -y cmake g++ > /dev/null
ROOT = "/content/flaw_worker"
if os.path.exists(os.path.join(ROOT, 'flaw', 'core')): ROOT = os.path.join(ROOT, 'flaw')
CORE = os.path.join(ROOT, "core")
BUILD = os.path.join(CORE, "build")
if os.path.exists(BUILD): shutil.rmtree(BUILD)
os.makedirs(BUILD, exist_ok=True)
os.chdir(BUILD)
!cmake ..
!make -j$(nproc)
libs = glob.glob("flaw_core*.so")
if libs:
    lib = os.path.abspath(libs[0])
    shutil.copy2(lib, os.path.join(ROOT, "flaw_core.so"))
    shutil.copy2(lib, os.path.join(ROOT, "ai", "flaw_core.so"))
    print("\n✅ BUILD READY!")

### 🏁 Step 3: Run
Replace YOUR_URL with your ngrok address.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"
import sys, os

# BRUTE FORCE PATH DETECTION
import os
found_root = "/content/flaw_worker"
for r, dirs, files in os.walk('/content/flaw_worker'):
    if 'ai' in dirs and 'core' in dirs:
        found_root = r
        break

print(f"Project root found at: {found_root}")
if found_root not in sys.path: sys.path.insert(0, found_root)

from ai.selfplay_worker import DistributedWorker
print("🚀 Starting worker...")
worker = DistributedWorker(MASTER_URL, depth=1)
worker.run()